# Persian Text Generation (RNN vs. Transformer)

This notebook runs the complete pipeline for the text generation project. It will:
1.  Install requirements.
2.  Download and preprocess the data.
3.  Train the RNN (GRU) model.
4.  Train the from-scratch Transformer model.
5.  Generate sample text with both models.
6.  Display the loss plots.

## 1. Setup and Data Preparation

In [ ]:
# Install requirements
!pip install -r requirements.txt

In [ ]:
# Download and unzip data using the script
# This requires your kaggle.json API key to be set up
!bash scripts/download_data.sh

In [ ]:
# Run preprocessing
!python scripts/preprocess.py \
    --input_file 'data/raw/Persian-WikiText-1.txt' \
    --stop_words_file 'data/raw/Persian_Stop_Words.txt' \
    --output_file 'data/processed/processed_text.txt' \
    --vocab_file 'data/processed/vocab.json' \
    --min_freq 5

## 2. Train RNN (GRU) Model

We will train the GRU model for 5 epochs. You can increase this for better results.

In [ ]:
!python scripts/train.py \
    --model_type rnn \
    --data_file 'data/processed/processed_text.txt' \
    --vocab_file 'data/processed/vocab.json' \
    --model_save_dir 'outputs/models/' \
    --plot_save_dir 'outputs/plots/' \
    --log_file 'logs/train_rnn.log' \
    --n_gram 3 \
    --batch_size 128 \
    --epochs 5 \
    --lr 0.001 \
    --embed_size 128 \
    --hidden_size 256 \
    --num_layers 2 \
    --dropout 0.3

## 3. Train Transformer Model

Now we train the from-scratch Transformer. We use a smaller n-gram context (n=2) and fewer layers to manage training time. Transformer models are data-hungry and benefit from longer training.

In [ ]:
!python scripts/train.py \
    --model_type transformer \
    --data_file 'data/processed/processed_text.txt' \
    --vocab_file 'data/processed/vocab.json' \
    --model_save_dir 'outputs/models/' \
    --plot_save_dir 'outputs/plots/' \
    --log_file 'logs/train_transformer.log' \
    --n_gram 2 \
    --batch_size 128 \
    --epochs 5 \
    --lr 0.0005 \
    --embed_size 128 \
    --num_layers 2 \
    --num_heads 4 \
    --d_ff 256 \
    --dropout 0.1

## 4. Generate Text

Let's see the results. We use Top-K sampling (k=10) to avoid repetition.

In [ ]:
print("--- Generating with RNN (GRU) Model ---")
!python scripts/generate.py \
    --model_type rnn \
    --model_path 'outputs/models/rnn_best.pth' \
    --vocab_file 'data/processed/vocab.json' \
    --seed_text 'ویکی پدیا یک دانشنامه' \
    --n_gram 3 \
    --max_length 30 \
    --top_k 10 \
    --embed_size 128 \
    --hidden_size 256 \
    --num_layers 2

In [ ]:
print("--- Generating with Transformer Model ---")
!python scripts/generate.py \
    --model_type transformer \
    --model_path 'outputs/models/transformer_best.pth' \
    --vocab_file 'data/processed/vocab.json' \
    --seed_text 'ویکی پدیا یک دانشنامه' \
    --n_gram 2 \
    --max_length 30 \
    --top_k 10 \
    --embed_size 128 \
    --num_layers 2 \
    --num_heads 4 \
    --d_ff 256

## 5. View Loss Plots

In [ ]:
from IPython.display import Image, display

print("RNN (GRU) Model Loss")
display(Image(filename='outputs/plots/rnn_loss.png'))

print("Transformer Model Loss")
display(Image(filename='outputs/plots/transformer_loss.png'))